# 加载数据

In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('C:/Users/admin/mnist/', one_hot=True)

Extracting C:/Users/admin/mnist/train-images-idx3-ubyte.gz
Extracting C:/Users/admin/mnist/train-labels-idx1-ubyte.gz
Extracting C:/Users/admin/mnist/t10k-images-idx3-ubyte.gz
Extracting C:/Users/admin/mnist/t10k-labels-idx1-ubyte.gz


# 占位符

In [11]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])
x_image = tf.reshape(x, [-1,28,28,1])#把图片reshape成28*28
keep_prob = tf.placeholder(tf.float32)

# 构建网络

In [13]:
#卷积层
conv1=tf.layers.conv2d(inputs=x_image,  filters=32,  kernel_size=[5, 5], padding="same", activation=tf.nn.relu,
      kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
#池化层
pool1=tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
#经过池化层“剪裁”28*28已经变成14*14
conv2=tf.layers.conv2d(inputs=pool1,  filters=64,  kernel_size=[5, 5], padding="same", activation=tf.nn.relu,
      kernel_initializer=tf.truncated_normal_initializer(stddev=0.01))
pool2=tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
#14*14 --7*7
re1 = tf.reshape(pool2, [-1, 7 * 7 * 64])
dense1 = tf.layers.dense(inputs=re1,      units=64,   activation=tf.nn.relu,
                      kernel_initializer=tf.truncated_normal_initializer(stddev=0.01),
                      kernel_regularizer=tf.contrib.layers.l2_regularizer(0.003))
dense1_drop = tf.nn.dropout(dense1, keep_prob)
logits= tf.layers.dense(inputs=dense1_drop,  units=10,   activation=None,
                        kernel_initializer=tf.truncated_normal_initializer(stddev=0.01),
                        kernel_regularizer=tf.contrib.layers.l2_regularizer(0.003))

# 定义loss，以及优化方法还有准确率

In [14]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=logits))
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
softmax_result = tf.nn.softmax(logits)
pred = tf.argmax(softmax_result, axis=1)
true = tf.argmax(y_, axis=1)
accuracy = tf.reduce_mean(tf.cast(tf.equal(pred, true), tf.float32))

# 网络初始化

In [15]:
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
sess.run(init)

# 训练

In [19]:
for i in range(1000):
    batch = mnist.train.next_batch(50)
    _, loss_, acu = sess.run([train_step, loss, accuracy], {x: batch[0], y_: batch[1], keep_prob:0.5})
    if i % 100 == 0:
        print("Step:{:>5}, Loss:{:>7.4f}, Accuracy:{:>7.2%}".format(i, loss_, acu))
loss_, acu = sess.run([loss, accuracy], {x: mnist.test.images, 
                                        y_: mnist.test.labels, keep_prob:1.0})
print("Training Finished, Loss:{:>7.4f}, Accuracy:{:>7.2%}".format(loss_, acu))

Step:    0, Loss: 0.0612, Accuracy:100.00%
Step:  100, Loss: 0.1981, Accuracy: 94.00%
Step:  200, Loss: 0.1268, Accuracy: 94.00%
Step:  300, Loss: 0.0946, Accuracy: 94.00%
Step:  400, Loss: 0.2122, Accuracy: 96.00%
Step:  500, Loss: 0.1571, Accuracy: 98.00%
Step:  600, Loss: 0.0903, Accuracy: 96.00%
Step:  700, Loss: 0.0638, Accuracy: 98.00%
Step:  800, Loss: 0.1046, Accuracy: 92.00%
Step:  900, Loss: 0.1035, Accuracy: 96.00%
Training Finished, Loss: 0.0804, Accuracy: 97.72%
